### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées en un tableau de bord [PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI) permettant d'avoir un portrait d'ensemble des données.  

---

In [7]:
import pandas as pd

from utils.SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.tools as tls

from rdflib import *

**Chargement des données**

In [8]:
data = updateInfoProfs()

**Expertises de recherche**

- Affiliations (facultés, départements, établissements affiliés)
- Disciplines
- Secteur de recherche
- Périodes chronologiques étudiées
- Régions géographiques / pays étudiés

In [9]:
expertises = data[['idsadvr', 'affiliations', 'etablissementsAffilies', 'expertise']]

toNormalize = ['affiliations', 'etablissementsAffilies', 'expertise', 'expertise.secteursRecherche',
                'expertise.disciplines', 'expertise.pays', 'expertise.continents', 'expertise.periodesChronologiques']

for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble', 'affiliations.fonction.codeSad', 
        'affiliations.fonction.nom', 'affiliations.local', 'affiliations.exclusion', 'affiliations.exclusionTel',
        'affiliations.uniteAdministrative.codeSad', 'affiliations.uniteAdministrative.nom', 'affiliations.telephone.numero',
        'affiliations.telephone.poste', 'expertise.phraseCle']

expertises = expertises.drop(columns=drop)

*Établissements affiliés*

In [10]:
def groupEtablissements(etablissementNom : str) -> str : 
   return etablissementNom.split(' – ')[0]

# Etablissements affiliés
etablissementsAffilies = expertises.dropna(subset='etablissementsAffilies.nom')
etablissementsAffilies = etablissementsAffilies.drop_duplicates(subset=(['idsadvr', 'etablissementsAffilies.nom']))
etablissementsAffilies.loc[:, 'etablissementsAffilies.nom'] = etablissementsAffilies['etablissementsAffilies.nom'].apply(lambda x : x.split(' – ')[0])
etablissementsAffilies = pd.DataFrame(plotVariable(etablissementsAffilies, 'etablissementsAffilies.nom')).sort_values(by='count', ascending = True)
#etablissementsAffilies = groupOtherValues(etablissementsAffilies, 10).reset_index(drop=True)

etablissementsAffilies.to_csv('tables/statistiques__expertisesRecherche/etablissementsAffiliesPie.csv', index=False)
etablissementsAffilies

,labels,count
16,Institut Lady Davis de de recherche médicale (...,1
14,Centre de recherche sur la dynamique du systèm...,1
15,CIUSS de la Mauricie-et-du-Centre-du-Québec,1
13,Centre interuniversitaire de recherche sur la ...,3
12,Centre de recherche interdisciplinaire en réad...,5
11,CIUSS du Centre-Ouest-de-l’Île-de-Montréal,5
10,CISSS Montérégie-Centre,6
9,Institut national de santé publique du Québec ...,7
8,Institut Philippe-Pinel de Montréal (IPP),8
7,CISSS de Laval,9


In [43]:
etablissementsAffilies['noms'] = etablissementsAffilies['labels'].apply(lambda x: (str(x)[:40] + "..."))
figEtablissementsAffilies = px.pie(
    etablissementsAffilies,
    labels = etablissementsAffilies['labels'],
    names = etablissementsAffilies['noms'],
    values = etablissementsAffilies['count'],
    title = 'Nombre de professeurs par établissements affiliés',
    hole = 0.6
)

figEtablissementsAffilies.update_traces(
    textposition='inside')

figEtablissementsAffilies.update_layout(
    uniformtext_minsize=12, 
    uniformtext_mode='hide'
)

**Facultés, départements et établissements affiliés**

*Nombre de professeurs par facultés*

In [12]:
facultes = pd.DataFrame(plotVariable(expertises, 'affiliations.faculte.nom'))[:-2].sort_values(by='count')
facultes.to_csv('tables/statistiques__expertisesRecherche/profsParFaculte.csv', index=False)

px.pie(
    facultes, 
    values= facultes['count'], 
    names= facultes['labels'], 
    title='Nombre de professeur-e-s par facultés',
    hole=0.6 
)

*Nombre de professeurs par discipline de recherche*

In [13]:
disciplines = expertises[['idsadvr', 'expertise.disciplines.nom', 
                        'expertise.disciplines.uid', 'expertise.disciplines.codeLangue']].dropna(subset='expertise.disciplines.uid').drop_duplicates()

disciplines = disciplines.groupby(['expertise.disciplines.nom', 'expertise.disciplines.uid', 
                    'expertise.disciplines.codeLangue'])['idsadvr'].count().reset_index().rename(columns={'idsadvr': 'count'})

disciplines = disciplines.sort_values(by=['expertise.disciplines.uid', 'expertise.disciplines.codeLangue'], ascending=[True, False])
disciplines = disciplines.drop_duplicates(subset='expertise.disciplines.uid', keep='first').sort_values(by='count', ascending=False)

disciplines = disciplines[['expertise.disciplines.nom', 'count']]

disciplines = groupOtherValues(disciplines, 30)[:30]
disciplines.to_csv('tables/statistiques__expertisesRecherche/disciplines_top30.csv', index=False)
disciplines

,expertise.disciplines.nom,count
270,Neurosciences,196
369,Sociologie,194
337,Psychologie,180
52,Biologie moléculaire,179
358,Santé publique,167
405,Épidémiologie et biostatistique,122
193,Informatique,121
312,Pharmacologie,121
178,Histoire,114
360,Science politique,110


In [14]:
fig = go.Figure(go.Treemap(
    labels= disciplines['expertise.disciplines.nom'],
    parents= [''] * len(disciplines),
    values = disciplines['count']
))
fig.update_layout(title_text="30 principales disciplines de recherche à l'UdeM")

**Expertises de recherche: Principales disciplines de recherche par faculté**

In [15]:
mappingDisciplines = pd.read_csv('tables/SADVR_disciplines.csv')
mappingDisciplines = mappingDisciplines[mappingDisciplines['noms.codeLangue'] == 'fre']
mappingDisciplines = {str(x['id']): x['noms.nom'] for x in mappingDisciplines.to_dict('records')}

In [16]:
disciplines = expertises[['idsadvr', 'affiliations.faculte.nom', 'expertise.disciplines.uid']].dropna(subset='expertise.disciplines.uid').drop_duplicates()
disciplines['expertise.disciplines.nom'] = disciplines['expertise.disciplines.uid'].astype(str).map(mappingDisciplines)
disciplines = disciplines.drop(columns='expertise.disciplines.uid')

# Group by 'faculty' and 'discipline' and count the number of professors in each group
faculty_discipline_counts = disciplines.groupby(['affiliations.faculte.nom', 'expertise.disciplines.nom'])['idsadvr'].count().reset_index()

# Rename the 'id' column to 'professor_count' for clarity
faculty_discipline_counts = faculty_discipline_counts.rename(columns={'idsadvr': 'count'})
faculty_discipline_counts = faculty_discipline_counts.sort_values(by='count', ascending=False)

#faculty_discipline_counts.to_csv('tables/statistiques__expertisesRecherche/disciplinesParFaculte.csv', index=False)
faculty_discipline_counts = faculty_discipline_counts.sort_values(by='count', ascending=False)

In [17]:
def groupOtherValues(df: pd.DataFrame, threshold: int = 6) -> pd.DataFrame:
    """
    Cette fonction prend en paramètre un objet DataFrame contenant une distirbution de fréquences et un nombre entier 
    représentant le nombre de valeurs à représenter dans un graphique. Elle retourne le DataFrame modifié en regroupant toutes 
    les autres valeurs dans une catégorie "Autre", permettant ainsi d'alléger la visualisaiton en réduisant le nombre de 
    catégories qui seornt affichées dans une visualisation.

    Par défaut, la fonction prend les 6 principales valeurs et groupe les autres dans la catégorie "Autre".
    
    À noter que la colonne contenant les fréquences associées aux catégories doit s'appeler 'count'.
    """
    top_values = df.head(threshold)
    other_values_count = df[threshold:]['count'].sum()
    col = df.columns[0]
    other_values = pd.DataFrame({col: ['Autre'], 'count': [other_values_count]})
    
    return pd.concat([top_values, other_values])

def generate_pie_chart(selected_faculty):
    filtered_df = faculty_discipline_counts[faculty_discipline_counts['affiliations.faculte.nom'] == selected_faculty]

    # Extraire les dix principales disciplines associées à une faculté
    filtered_df = filtered_df.sort_values(by='count', ascending=False)
    filtered_df = groupOtherValues(filtered_df, 6)[:6]
    fig = go.Figure(go.Pie(
        labels= filtered_df['expertise.disciplines.nom'], 
        values = filtered_df['count'],
        hole = 0.6)
    )
    return fig

# Create a figure for each category
figs = {
    c: generate_pie_chart(c).update_traces(name=c, visible=False)
    for c in faculty_discipline_counts['affiliations.faculte.nom'].unique()
}

fig = go.Figure()

# Default category
defaultcat = faculty_discipline_counts['affiliations.faculte.nom'].unique()[0]
fig.add_traces(figs[defaultcat].data).update_traces(visible=True)

# integrate figures per category into one figure
for k in figs.keys():
    if k != defaultcat:
        fig.add_traces(figs[k].data)

# finally build dropdown menu
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": k,
                    "method": "update",
                    
                    # list comprehension for which traces are visible
                    "args": [{"visible": [kk == k for kk in figs.keys()]}],
                }
                for k in figs.keys()
            ]
        }
    ]
).show()

In [18]:
fig.write_html("visualisations/disciplinesFacultes.html")

**Expertises de recherche: cartographie des expertises par mots-clés**

- Mots-clés / Phases clés associés aux disciplines / facultés / départements de recherche 

Voir si on peut extraire un graphe et le visualiser

*Disciplines*

In [19]:
disciplines = expertises[[x for x in expertises.columns if 'discipline' in x or 'motsCles' in x or 'phraseCle' in x or x == 'idsadvr']]
disciplines

,idsadvr,expertise.motsCles,expertise.disciplines.uid,expertise.disciplines.codeLangue,expertise.disciplines.nom,expertise.disciplines.ordre
0,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",19,fre,Biochimie,2
1,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",20,fre,Biologie cellulaire,1
2,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",22,fre,Biologie moléculaire,3
3,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",19,eng,Biochemistry,2
4,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",20,eng,Cell Biology,1
...,...,...,...,...,...,...
285318,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",77,eng,Computer Science,1
285319,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",78,eng,Administrative Computing,2
285320,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",83,eng,Management,3
285321,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",86,eng,Pure Mathematics,4


In [20]:
freqDisciplines = pd.DataFrame(plotVariable(disciplines, 'expertise.disciplines.uid', mapping=mappingDisciplines))
freqDisciplines

,labels,count
0,Neurosciences,196
1,Sociologie,194
2,Psychologie,180
3,Biologie moléculaire,179
4,Santé publique,167
...,...,...
201,Oncologie gynécologique,1
202,Génie industriel,1
203,Mycologie,1
204,Anatomie,1


In [21]:
# 1 premier exemple: Neurosciences
# Une fois que ça aura fonctionné pour une discipline, on va créer une fonction qui pourra le faire pour toutes les disciplines
neuro = disciplines[disciplines['expertise.disciplines.uid'] == '98']
neuro = explodeNormalize(neuro, 'expertise.motsCles')
neuro = neuro.dropna(subset='expertise.motsCles.nom')
neuro = neuro[neuro['expertise.motsCles.ordre'].astype(int) <= 5]

neuro = neuro.drop_duplicates(subset=['idsadvr', 'expertise.motsCles.uid'])
neuro = neuro [['idsadvr',	'expertise.disciplines.uid', 'expertise.disciplines.nom', 'expertise.motsCles.uid',
                'expertise.motsCles.nom',	'expertise.motsCles.ordre',	'expertise.motsCles.codeLangue']]

freqMotsCles = plotVariable(neuro, 'expertise.motsCles.uid').to_dict('records')
freqMotsCles = {x['expertise.motsCles.uid'] : x['count'] for x in freqMotsCles}

neuro['expertises.motCles.count'] = neuro['expertise.motsCles.uid'].map(freqMotsCles)
neuro = neuro.drop_duplicates(subset = ['expertise.motsCles.uid']).drop(columns='idsadvr')

neuro = neuro.sort_values(by='expertises.motCles.count', ascending=False).reset_index(drop=True)
neuro

AttributeError: 'dict' object has no attribute 'to_dict'

*On va essayer de créer un graphe avec*

In [ ]:
graph = neuro[:30]
listeMotsCles = graph.to_dict('records')

idDiscipline = str(98)

uriDisciplines = URIRef(f'{baseURI}/disciplines')
uriDiscipline = URIRef(f'{baseURI}/disciplines/{idDiscipline}')
uriCount = URIRef(f'{baseURI}/count')
uriHasKW = URIRef(f'{baseURI}/hasKeyword')

g = Graph()
g.bind('sadvr', 'https://www.recherche.umontreal.ca/vitrine/rest/api/1.7/umontreal/')

# Classe pour les disciplines
g.add((uriDisciplines, RDF.type, OWL.Class))
g.add((uriDisciplines, RDFS.label, Literal('Discipline', lang='fr')))

# Classe et nom de la discipline (fr)
g.add((uriDiscipline, RDF.type, uriDisciplines))
g.add((uriDiscipline, RDF.type, OWL.Class))
g.add((uriDiscipline, RDFS.label, Literal(mappingDisciplines[idDiscipline], lang='fr')))

# Relations
# keyWordCount
g.add((uriCount, RDF.type, OWL.ObjectProperty))
g.add((uriCount, RDFS.label, Literal('keywordCount', lang='en')))

for kw in listeMotsCles:
    id = kw['expertise.motsCles.uid']
    uri = URIRef(f'{baseURI}/motsCles/{id}')
    label = Literal(kw['expertise.motsCles.nom'])
    compte = Literal(kw['expertises.motCles.count'], datatype=XSD.integer)

    # Lien discipline <> mot-clé
    g.add((uri, RDFS.subClassOf, uriDiscipline))

    # Construction du noeud pour le mot-clé
    g.add((uri, RDF.type, OWL.Class))
    g.add((uri, RDFS.label, label))
    g.add((uri, uriCount, compte))

In [ ]:
g.serialize('visualisations/graph_test_neurosciences.ttl', format='ttl')

<Graph identifier=Nb23fa3f03cf24fe199089ea723b10c12 (<class 'rdflib.graph.Graph'>)>

**Visualiser le graphe**